In [1]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [2]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

# Import model
from architectures.pgnniv_baseline import PGNNIVBaseline

In [3]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/sigmoid_nonlinear/sigmoid_nonlinear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/transfer_learning')

MODEL_RESULTS_TRANSFERLEARNING_PATH = os.path.join(ROOT_PATH, r'results/transfer_learning/baseline')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_TRANSFERLEARNING_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/transfer_learning
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/transfer_learning/baseline


In [4]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/sigmoid_nonlinear/sigmoid_nonlinear.pkl


In [5]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [6]:
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda:1


In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
n_modes = 10

# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, n_modes, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [ ]:
# Load model and the optimizer
model = PGNNIVBaseline(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 100000

batch_size = 64
n_checkpoints = 3

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_TRANSFERLEARNING_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 6.968e+08, Test loss: 8.965e+08, MSE(e): 6.961e+01, MSE(pi1): 4.383e+01, MSE(pi2): 2.977e+01, MSE(pi3): 1.933e+00
Epoch 100, Train loss: 1.254e+07, Test loss: 1.425e+07, MSE(e): 1.235e+00, MSE(pi1): 1.119e+01, MSE(pi2): 8.481e-01, MSE(pi3): 8.273e-01
Epoch 200, Train loss: 1.243e+07, Test loss: 1.374e+07, MSE(e): 1.235e+00, MSE(pi1): 3.436e+00, MSE(pi2): 8.455e-01, MSE(pi3): 4.016e-01
Epoch 300, Train loss: 1.238e+07, Test loss: 1.376e+07, MSE(e): 1.230e+00, MSE(pi1): 3.707e+00, MSE(pi2): 8.444e-01, MSE(pi3): 4.182e-01
Epoch 400, Train loss: 1.235e+07, Test loss: 1.363e+07, MSE(e): 1.227e+00, MSE(pi1): 3.631e+00, MSE(pi2): 8.439e-01, MSE(pi3): 3.773e-01
Epoch 500, Train loss: 1.235e+07, Test loss: 1.361e+07, MSE(e): 1.227e+00, MSE(pi1): 3.635e+00, MSE(pi2): 8.444e-01, MSE(pi3): 4.427e-01
Epoch 600, Train loss: 1.234e+07, Test loss: 1.349e+07, MSE(e): 1.227e+00, MSE(pi1): 3.589e+00, MSE(pi2): 8.422e-01, MSE(pi3): 3.808e-01
Epoch 700, 

In [ ]:
# Parametros de entrenamiento
start_epoch = n_epochs-1
n_epochs = 150000

batch_size = 64 
n_checkpoints = 3

second_lr = 3e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_TRANSFERLEARNING_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 80.
Epoch 80, Train loss: 1.255e+07, Test loss: 1.404e+07, MSE(e): 1.229e+00, MSE(pi1): 1.474e+01, MSE(pi2): 8.448e-01, MSE(pi3): 1.110e+00
Epoch 90, Train loss: 1.254e+07, Test loss: 1.402e+07, MSE(e): 1.229e+00, MSE(pi1): 1.467e+01, MSE(pi2): 8.449e-01, MSE(pi3): 1.110e+00

Training process finished after 100 epochs

